In [1]:

"""
This file is containing a class, that perform all modules for 
Hovagim Bakardjian system that serves for feature extraction and command
classification in SSVEP based BCI. 
Version 2
"""

import numpy as np
import pandas as pd
from bieg import ICAManager
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.stats import mode


class BakardjianSystem(object):
    
    def __init__(self,path,seconds=1,components_to_exclude = [0,14,27,28,127],
                 chunk = 1,extract = False,freq = 256,classes = 'three_classes',normalize=True):

        self.path = path
        self.seconds = seconds
        self.ica_file = ICAManager(input_path=self.path,method='fastica',sep=' ')
        self.components_to_exclude = components_to_exclude
        self.extract = extract
        self.freq = freq
        self.classes = classes
        self.normalize = normalize
        
    def prep_data_extract_components(self):
        
        self.ica_file.load_data()
        
        if self.extract == True:
            self.ica_file.extract_components()
            self.ica_file.exclude_ica_components(
                components_to_exclude=self.components_to_exclude)
        
        self.data = self.ica_file.data
        self.o1 = self.data[15]
        self.oz = self.data[23]
        self.o2 = self.data[28]

    @staticmethod
    def filtering(data,low,high,freq):
        
        bplowcut = low/(freq*0.5)
        bphighcut = high/(freq*0.5) 

        [b,a] = sig.butter(N=4,Wn=[bplowcut,bphighcut],btype='bandpass')

        filtered = sig.filtfilt(b,a,data)
        return filtered
    
    def go(self,low,high):
        
        """This functon replaces variance analyzer and smoother, serves as both"""
        
        electrode = np.zeros(self.o1.shape[0])
        
        for n in [self.o1,self.oz,self.o2]:
            filtered = self.filtering(n,low,high,self.freq)
            var = np.abs(filtered)
            smoothed = sig.savgol_filter(var,polyorder=2,
                                         window_length=self.seconds)
            electrode += smoothed
            
        electrode = electrode/3
        
        return electrode
            
    def normalizer(self):
        
        band8hz = [7.9,8.1]
        band14hz = [13.9,14.1]
        band28hz = [27.9,28.1]
        
        if self.classes == 'two_classes':
            
            sig_8hz = self.go(band8hz[0],band8hz[1])
            sig_14hz = self.go(band14hz[0],band14hz[1])
            
            if self.normalize == True:
                self.sig_8hz = sig_8hz/(sig_8hz+sig_14hz)
                self.sig_14hz = sig_14hz/(sig_8hz+sig_14hz)
            else:
                self.sig_8hz = sig_8hz
                self.sig_14hz = sig_14hz
            
        elif self.classes == 'three_classes':
            
            sig_8hz = self.go(band8hz[0],band8hz[1])
            sig_14hz = self.go(band14hz[0],band14hz[1])
            sig_28hz = self.go(band28hz[0],band28hz[1])
            
            if self.normalize == True:
                self.sig_8hz = sig_8hz/(sig_8hz+sig_14hz+sig_28hz)
                self.sig_14hz = sig_14hz/(sig_8hz+sig_14hz+sig_28hz)
                self.sig_28hz = sig_28hz/(sig_8hz+sig_14hz+sig_28hz)
            else:
                self.sig_8hz = sig_8hz
                self.sig_14hz = sig_14hz
                self.sig_28hz = sig_28hz
            
    def classifier(self):
        
        classified = np.zeros(self.sig_8hz.shape)
        
        if self.classes == 'two_classes':
            for n in range(self.sig_8hz.shape[0]):
                dict_classes = {self.sig_8hz[n]:0,self.sig_14hz[n]:1}
                val_max = np.max(np.array([self.sig_8hz[n],self.sig_14hz[n]]))
                classified[n] = dict_classes[val_max]
        
        elif self.classes == 'three_classes':
            for n in range(self.sig_8hz.shape[0]):
                dict_classes = {self.sig_8hz[n]:0,self.sig_14hz[n]:1,self.sig_28hz[n]:2}
                val_max = np.max(np.array([self.sig_8hz[n],self.sig_14hz[n],
                                          self.sig_28hz[n]]))
                classified[n] = dict_classes[val_max]
        
        decision = int(mode(classified)[0][0])
        
        return decision          

if __name__ == '__main__':
    bs = BakardjianSystem(path= 'data28Hz_3_seconds/ssvep28Hz_sec3_prt0.csv',seconds=3,classes='three_classes')
    bs.prep_data_extract_components()
    plt.show()
    bs.normalizer()
    print(bs.classifier())
    

0


In [71]:
from sklearn.cross_decomposition import CCA

seconds = 3


bs = BakardjianSystem(path='data08Hz_3_seconds/ssvep08Hz_sec3_prt0.csv',seconds=seconds,normalize=False)
bs.prep_data_extract_components()
bs.normalizer()

class CCA_Classifier(object):
    
    """This class serves as classifier using Canonical Correlation Analysis
    for Bakardjian Method based system."""
    
    def __init__(self,path,seconds=3,freq = 256,classes = 'two_classes',extract = False):
        self = self
        self.seconds = seconds
        self.freq = freq
        self.classes = classes
        
        self.baksys = BakardjianSystem(path,seconds=seconds,normalize=False,classes=classes,extract=extract)
        self.baksys.prep_data_extract_components()
        self.baksys.normalizer()
        
    def reference(self,freq_ref = 8):
        
        """This function serves as creator of reference signal"""
        
        t = np.linspace(0,self.seconds,self.freq*self.seconds)
        
        reference = np.array(())
        
        sin = np.sin(2*np.pi*freq_ref*t).reshape(self.freq,self.seconds)
        cos = np.cos(2*np.pi*freq_ref*t).reshape(self.freq,self.seconds)
        
        return sin,cos
        
    def extract_correlation(self):
        
        if self.classes == 'two_classes':
            
            sig_8 = self.baksys.sig_8hz.reshape(self.freq,self.seconds)
            sig_14 = self.baksys.sig_14hz.reshape(self.freq,self.seconds)
            
            result = dict()
            
            for n in [sig_8,sig_14]:
                
                for z in [8,14]:
                    
                    sinref = self.reference(z)[0]
                    cosref = self.reference(z)[1]
                    
                    ccasin = CCA(n_components=1)
                    ccacos = CCA(n_components=1)
                    
                    ccasin.fit(n,sinref)
                    ccacos.fit(n,cosref)
                    
                    usin,vsin = ccasin.transform(n,sinref)
                    ucos,vcos = ccacos.transform(n,cosref)
                    
                    sincor = np.corrcoef(usin.T,vsin.T)[0,1]
                    coscor = np.corrcoef(ucos.T,vcos.T)[0,1]
                    
                    maxcor = abs(max([sincor,coscor]))
                    
                    result[maxcor] = z 
                
            self.corr_dict = result
        
        if self.classes == 'three_classes':
            
            sig_8 = self.baksys.sig_8hz.reshape(self.freq,self.seconds)
            sig_14 = self.baksys.sig_14hz.reshape(self.freq,self.seconds)
            sig_28 = self.baksys.sig_28hz.reshape(self.freq,self.seconds)
            
            result = dict()
            
            for n in [sig_8,sig_14,sig_28]:
                
                for z in [8,14,28]:
                    
                    sinref = self.reference(z)[0]
                    cosref = self.reference(z)[1]
                    
                    ccasin = CCA(n_components=1)
                    ccacos = CCA(n_components=1)
                    
                    ccasin.fit(n,sinref)
                    ccacos.fit(n,cosref)
                    
                    usin,vsin = ccasin.transform(n,sinref)
                    ucos,vcos = ccacos.transform(n,cosref)
                    
                    sincor = np.corrcoef(usin.T,vsin.T)[0,1]
                    coscor = np.corrcoef(ucos.T,vcos.T)[0,1]
                    
                    maxcor = abs(max([sincor,coscor]))
                    
                    result[maxcor] = z
                
            self.corr_dict = result
            
    def predict(self):
        
        corrdic = self.corr_dict
        res = corrdic[max(corrdic.keys())]
        
        return res

cca = CCA_Classifier(path='data14Hz_5_seconds/ssvep14Hz_sec5_prt2.csv',seconds=5,classes='two_classes',
                    extract=True)
uno = cca.extract_correlation()
print(cca.corr_dict)
cca.predict()



Creating RawArray with float64 data, n_channels=128, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 1690 samples (6.602 sec) selected
filter_length (1691) is longer than the signal (1280), distortion is likely. Reduce filter length or filter a longer signal.


/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  self.raw.filter(filter[0], filter[1], n_jobs=2)
/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: RuntimeWarning: filter_length (1691) is longer than the signal (1280), distortion is likely. Reduce filter length or filter a longer signal.
  self.raw.filter(filter[0], filter[1], n_jobs=2)


<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>
Fitting ICA to data using 128 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.


[Parallel(n_jobs=2)]: Done 128 out of 128 | elapsed:    0.1s finished


Using all PCA components: 128
<ICA  |  raw data decomposition, fit (fastica): 342 samples, 128 components, channels used: "eeg">
Creating RawArray with float64 data, n_channels=128, n_times=1280
    Range : 0 ... 1279 =      0.000 ...     4.996 secs
Ready.
Setting up band-pass filter from 1 - 40 Hz
fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 1690 samples (6.602 sec) selected
filter_length (1691) is longer than the signal (1280), distortion is likely. Reduce filter length or filter a longer signal.


/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: DeprecationWarning: fir_design defaults to "firwin2" in 0.15 but will change to "firwin" in 0.16, set it explicitly to avoid this warning.
  self.raw.filter(filter[0], filter[1], n_jobs=2)
/home/stakar/Studia/kogni/Magisterka/code/bieg.py:98: RuntimeWarning: filter_length (1691) is longer than the signal (1280), distortion is likely. Reduce filter length or filter a longer signal.
  self.raw.filter(filter[0], filter[1], n_jobs=2)


<ICA  |  no decomposition, fit (fastica):  samples, no dimension reduction>
Fitting ICA to data using 128 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.


[Parallel(n_jobs=2)]: Done 128 out of 128 | elapsed:    0.1s finished


Using all PCA components: 128
<ICA  |  raw data decomposition, fit (fastica): 342 samples, 128 components, channels used: "eeg">
Transforming to ICA space (128 components)
Zeroing out 5 ICA components
{0.022176148944422942: 8, 0.046129888586487339: 14, 0.046392096956317727: 8, 0.018309453142081676: 14}


8

In [97]:



U = np.random.random_sample(500).reshape(100,5)
V = np.random.random_sample(500).reshape(100,5)

cca = CCA(n_components=1)
cca.fit(U, V)

cca.coef_.shape                   # (5,5)

U_c, V_c = cca.transform(U, V)

U_c.shape                         # (100,1)
V_c.shape                         # (100,1)


cca = CCA(n_components=1)
U_c, V_c = cca.fit_transform(U, V)

result = np.corrcoef(U_c.T, V_c.T)[0,1]

In [ ]:

# U1 = bs.sig_8hz.reshape(bs.freq,bs.seconds)

# U2 = bs.sig_14hz.reshape(bs.freq,bs.seconds)

# t = np.linspace(0,seconds,bs.freq*seconds)

# refsin = np.sin(2*np.pi*8*t).reshape(bs.freq,bs.seconds)
# refcos = np.cos(2*np.pi*8*t).reshape(bs.freq,bs.seconds)

# ccasin = CCA(n_components=1)
# ccacos = CCA(n_components=1)

# ccasin.fit(U1,refsin)
# ccacos.fit(U1,refcos)

# usin,vsin = cca.transform(U1,refsin)
# ucos,vcos = cca.transform(U1,refcos)


# Usinmax = np.max(cca.coef_)

# cca.fit(U1,ref)
# U2sinmax = np.max(cca.coef_)

# Ucos = cca.fit(U,refcos)
# Ucosmax = np.max(Ucos.coef_)

# U2cos = cca.fit(U2,refcos)
# U2cosmax = np.max(U2cos.coef_)

# Ucosmax == U2cosmax

# np.corrcoef(u.T,v.T)[0,1]
